In [2]:
# LSTM RNN Stock Predictor Using Fear and Greed Index
# 10 day window of Bitcoin fear and greed index values to predict the 11th day closing price




In [39]:
import numpy as np
import pandas as pd
import hvplot.pandas

In [40]:
# Set the random seed for reproducibility
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)

In [41]:
# Load the fear and greed sentiment data for Bitcoin
fng_df=pd.read_csv('btc_sentiment.csv', index_col="date", infer_datetime_format=True, parse_dates=True)
fng_df=fng_df.drop(columns="fng_classification")
fng_df

,fng_value
date,
2019-07-29,19
2019-07-28,16
2019-07-27,47
2019-07-26,24
2019-07-25,42
...,...
2018-02-05,11
2018-02-04,24
2018-02-03,40


In [42]:
# Load the historical closing prices for Bitcoin
historical_df=pd.read_csv('btc_historic.csv', index_col="Date", infer_datetime_format=True, parse_dates=True)['Close']
historical_df2 =historical_df.sort_index()
historical_df.head()

Date
2018-01-02    14754.129883
2018-01-03    15156.620117
2018-01-04    15180.080078
2018-01-05    16954.779297
2018-01-06    17172.300781
Name: Close, dtype: float64

In [43]:
df=fng_df.join(historical_df, how="inner")
df.head()

,fng_value,Close
2018-02-01,30,9114.719727
2018-02-02,15,8870.820313
2018-02-03,40,9251.269531
2018-02-04,24,8218.049805
2018-02-05,11,6937.080078


In [44]:
df.tail()

,fng_value,Close
2019-07-25,42,9882.429688
2019-07-26,24,9847.450195
2019-07-27,47,9478.320313
2019-07-28,16,9531.769531
2019-07-29,19,9529.889648


In [45]:
def window_data(df, window, feature_col_number, target_col_number):
    X=[]
    y=[]
    for i in range(len(df)-window-1):
        features=df.iloc[i:(i+window), feature_col_number]
        target=df.iloc[(i+window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1,1)

In [46]:
window_data

<function __main__.window_data(df, window, feature_col_number, target_col_number)>

In [47]:
# Predict Closing Prices using a 10 day window of previous fng values
# Then, experiment with window size = 10
window=10

# Column index 0 is the 'fng_value' column
# Column index 1 is the `Close` column
feature_column=0
target_column=0
X, y = window_data(df, window, feature_column, target_column)

In [48]:
# Use 70% of the data for training and the remaineder for testing
split = int(len(X)*0.7)
split

372

In [49]:
X_train=X[:split-1]
X_test=X[split:]
y_train=y[:split-1]
y_test=y[split:]

In [50]:
from sklearn.preprocessing import MinMaxScaler
x_train_scaler=MinMaxScaler()
x_test_scaler=MinMaxScaler()
y_train_scaler=MinMaxScaler()
y_test_scaler=MinMaxScaler()

x_train_scaler.fit(X_train)
y_train_scaler.fit(y_train)

X_train=x_train_scaler.transform(X_train)
y_train=y_train_scaler.transform(y_train)

In [51]:
x_test_scaler.fit(X_test)
y_test_scaler.fit(y_test)

X_test=x_test_scaler.transform(X_test)
y_test=y_test_scaler.transform(y_test)

In [52]:
# Reshape the features for the model
X_train=X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test=X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

---

In [53]:
# Build and fit a custom LSTM RNN

In [54]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [55]:
model=Sequential()
model.add(LSTM(units=30, return_sequences=True,
              input_shape=(X_train.shape[1],1)))
model.add(Dropout(0.2))
model.add(LSTM(units=30, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=30))
model.add(Dropout(0.2))
model.add(Dense(1))

In [56]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [57]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 10, 30)            3840      
_________________________________________________________________
dropout_3 (Dropout)          (None, 10, 30)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 10, 30)            7320      
_________________________________________________________________
dropout_4 (Dropout)          (None, 10, 30)            0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 30)                7320      
_________________________________________________________________
dropout_5 (Dropout)          (None, 30)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                

In [58]:
# Train the model
model.fit(X_train, y_train, epochs=10, shuffle=False, batch_size=1, verbose=1)

Epoch 1/10
371/371 [==============================] - 5s 6ms/step - loss: 0.0872
Epoch 2/10
371/371 [==============================] - 2s 6ms/step - loss: 0.0545
Epoch 3/10
371/371 [==============================] - 2s 6ms/step - loss: 0.0566
Epoch 4/10
371/371 [==============================] - 2s 6ms/step - loss: 0.0532
Epoch 5/10
371/371 [==============================] - 2s 6ms/step - loss: 0.0449
Epoch 6/10
371/371 [==============================] - 2s 6ms/step - loss: 0.0359
Epoch 7/10
371/371 [==============================] - 2s 6ms/step - loss: 0.0227
Epoch 8/10
371/371 [==============================] - 2s 6ms/step - loss: 0.0256
Epoch 9/10
371/371 [==============================] - 2s 6ms/step - loss: 0.0212
Epoch 10/10
371/371 [==============================] - 2s 6ms/step - loss: 0.0218


---

In [25]:
# Evaluate model performance
# Apply the "inverse_transform" function to the predicted and y_test values

In [59]:
model.evaluate(X_test, y_test)

5/5 [==============================] - 1s 3ms/step - loss: 0.0253


0.025336792692542076

In [60]:
predicted = model.predict(X_test)

In [61]:
# Recover the original prices instead of the scaled version
predicted_prices=y_test_scaler.inverse_transform(predicted)
real_prices=y_test_scaler.inverse_transform(y_test.reshape(-1, 1))

In [62]:
# Create a DataFrame of Real and Predicted values
btc_prices=pd.DataFrame({
    "Real prices": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()}, 
    index=df.index[-len(real_prices): ]) 
btc_prices


,Real prices,Predicted
2019-02-20,65.0,47.672737
2019-02-21,59.0,56.842693
2019-02-22,59.0,58.231400
2019-02-23,61.0,56.171398
2019-02-24,63.0,54.605541
...,...,...
2019-07-25,20.0,43.123512
2019-07-26,42.0,36.022869
2019-07-27,24.0,36.003807
2019-07-28,47.0,33.589191


In [63]:
btc_prices.hvplot()

:NdOverlay   [Variable]
   :Curve   [index]   (value)